In [1]:
import pandas as pd
import datetime
from gsheets_connect import GoogleSheets, Connect

In [2]:
conn = Connect(service='google_sheets')

In [3]:
conn.get_service()

In [4]:
gsheet = GoogleSheets()

In [5]:
service = gsheet.service

In [51]:
def numberToLetters(q):
    q = q - 1
    result = ''
    while q >= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

In [84]:
cur_data = pd.read_csv(r'daily_data.txt', sep=' ')

#cur_data['Payment Day'] = pd.to_datetime(cur_data['Payment Day']).dt.date

cur_data['Payment Fortnight'] = cur_data['Payment Day'].apply(
    lambda date: str(pd.to_datetime(date).replace(day=15 if pd.to_datetime(date).day <= 15 else 30).date())
)

In [85]:
cur_data

,Amount $,Expense Day,Expense Month,Payment Day,Payment Month,Installments,Payment Method,Method Name,Category,Sub Category,Payment Fortnight
0,267.000000,2019-08-01,201908.0,2019-08-01,201908.0,1.0,debit,debit,Food,Restaurants,2019-08-15
1,633.000000,2019-08-12,201908.0,2019-08-15,201908.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-08-15
2,633.000000,2019-08-12,201908.0,2019-09-15,201909.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-09-15
3,633.000000,2019-08-12,201908.0,2019-10-15,201910.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-10-15
4,633.000000,2019-08-12,201908.0,2019-11-15,201911.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-11-15
5,633.000000,2019-08-12,201908.0,2019-12-15,201912.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2019-12-15
6,633.000000,2019-08-12,201908.0,2020-01-15,202001.0,1.0,credit,Santander-Free,Personal Stuff,Stuff for me,2020-01-15
7,625.166667,2019-08-14,201908.0,2019-08-30,201908.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-08-30
8,625.166667,2019-08-14,201908.0,2019-09-30,201909.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-09-30
9,625.166667,2019-08-14,201908.0,2019-10-30,201910.0,1.0,credit,Bancomer-Platinum,Personal Stuff,Stuff for me,2019-10-30


In [86]:
cols = np.array(cur_data.columns)

cols_values = cols.tolist()

values_arr = cur_data.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [87]:
list_of_rows

array([['Amount $', 'Expense Day', 'Expense Month', 'Payment Day',
        'Payment Month', 'Installments', 'Payment Method', 'Method Name',
        'Category', 'Sub Category', 'Payment Fortnight'],
       [267.0, '2019-08-01', 201908.0, '2019-08-01', 201908.0, 1.0,
        'debit', 'debit', 'Food', 'Restaurants', '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-08-15', 201908.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-09-15', 201909.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-09-15'],
       [633.0, '2019-08-12', 201908.0, '2019-10-15', 201910.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-10-15'],
       [633.0, '2019-08-12', 201908.0, '2019-11-15', 201911.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-11-15'],
       [633.0, '2019-08-1

In [88]:
data = [
    {
        'range': 'data!A1',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

154 cells updated.


In [7]:
monthly_data=cur_data.groupby(
    ['Payment Fortnight','Category', 'Sub Category']
).agg({'Amount $':'sum'})

In [89]:
monthly_data#.unstack(0)

Amount $
Payment Fortnight Category       Sub Category            
2019-08-15        Food           Restaurants   267.000000
                  Personal Stuff Stuff for me  633.000000
2019-08-30        Personal Stuff Stuff for me  625.166667
2019-09-15        Personal Stuff Stuff for me  633.000000
2019-09-30        Personal Stuff Stuff for me  625.166667
2019-10-15        Personal Stuff Stuff for me  633.000000
2019-10-30        Personal Stuff Stuff for me  625.166667
2019-11-15        Personal Stuff Stuff for me  633.000000
2019-11-30        Personal Stuff Stuff for me  625.166667
2019-12-15        Personal Stuff Stuff for me  633.000000
2019-12-30        Personal Stuff Stuff for me  625.166667
2020-01-15        Personal Stuff Stuff for me  633.000000
2020-01-30        Personal Stuff Stuff for me  625.166667

In [55]:
monthly_data.unstack(0).shape

(2, 12)

In [58]:
numberToLetters(12)

'L'

In [ ]:
rows_range = 

In [39]:
dates = np.array(monthly_data.unstack(0).columns.levels[1])

cols_values = np.array(['category', 'sub_category']+dates.astype(str).tolist())

values_arr = monthly_data.unstack(0).fillna(0).reset_index().values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [59]:
list_of_rows.shape

(3, 14)

In [71]:
start_from = 'biweekly pivot!B2'

In [66]:
table_range

'B2:N3'

In [72]:
data = [
    {
        'range': start_from,
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

42 cells updated.


In [9]:
list_of_rows

[['Food',
  'Restaurants',
  267.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 ['Personal Stuff',
  'Stuff for me',
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666]]